# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [11]:
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import spacy
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import nltk

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

In [12]:
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [14]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Лемматизация

In [16]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def spacy_lemm(row):
    doc = nlp(row)  
    lemma = ' '.join([token.lemma_ for token in doc])
    lemma = ''.join(re.sub(r'[^A-Za-z]',' ',lemma))
    lemma = " ".join(lemma.split())
    return lemma

In [17]:
df['lemm']=df['text'].apply(spacy_lemm)

Разделим на тренировочную и тестову выборки

In [18]:
target = df['toxic']
features = df['lemm']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

Токенизация

In [19]:
nltk.download('stopwords')
stop_words_my = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
corpus_train = features_train
corpus_valid = features_valid

In [21]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stop_words_my) 

tf_idf_train = count_tf_idf.fit_transform(corpus_train) 
tf_idf_valid = count_tf_idf.transform(corpus_valid)

print("Размер матрицы train:", tf_idf_train.shape)
print("Размер матрицы train:", tf_idf_valid.shape)

Размер матрицы train: (127433, 137317)
Размер матрицы train: (31859, 137317)
CPU times: user 6.68 s, sys: 95.6 ms, total: 6.78 s
Wall time: 6.93 s


## Обучение

In [22]:
cv = KFold(n_splits=3, shuffle=True, random_state=12345)

LogisticRegression 

In [23]:
%%time

model_lr_bal = LogisticRegression(class_weight='balanced')

train_f1 = cross_val_score(model_lr_bal, 
                      tf_idf_train, 
                      target_train, 
                      cv=cv, 
                      scoring='f1')

print('F1 на CV', train_f1.mean())

F1 на CV 0.7479530151518589
CPU times: user 32.2 s, sys: 59.9 s, total: 1min 32s
Wall time: 1min 32s


DecisionTree 

In [17]:
%%time

model_dt = DecisionTreeClassifier()
params = [{'max_depth':[10,30,50,100],
                'random_state':[12345],
                'class_weight':['balanced']}]

gridsearch = GridSearchCV(model_dt, params, scoring='f1',cv=cv)
gridsearch.fit(tf_idf_train, target_train)
print(gridsearch.best_params_)

{'class_weight': 'balanced', 'max_depth': 100, 'random_state': 12345}
CPU times: user 6min 25s, sys: 636 ms, total: 6min 26s
Wall time: 6min 26s


In [18]:
gridsearch.best_score_

0.6425070378296799

Модель LogisticRegression показала себя лучше. Проверим ее на тестовой выборке

In [16]:
model_lr_bal.fit(tf_idf_train, target_train)

prediction_lr_valid = model_lr_bal.predict(tf_idf_valid)

print('F1 valid', f1_score(target_valid, prediction_lr_valid))

F1 valid 0.7535801134828425


## Выводы

Удалось достичь метрики F1 0.7535 на валидационном наборе данных, выполнив лемматизацию.